In [1]:
!pip install gymnasium
!pip install git+https://github.com/pablocarpon/rSoccer.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 9.5 MB/s eta 0:00:00
  Cloning https://github.com/pablocarpon/rSoccer.git to /tmp/pip-req-build-_5ppqcfs
  Running command git clone --filter=blob:none --quiet https://github.com/pablocarpon/rSoccer.git /tmp/pip-req-build-_5ppqcfs
  Resolved https://github.com/pablocarpon/rSoccer.git to commit bf8b88bb228835b67aba3f3b967624a12aba226a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.6/601.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.4 MB/s eta 0:00:00
  Created wheel for rsoccer-gym: filename=rsoccer_gym-0.1.dev517+gbf8b88b-py3-none-any.whl size=37623 sha256=f5bc9a7c2f54b797693f0a8d90055f36142cb299fa97a887aa0a3bea4c544779
  Stored in directory: /tmp/pip-ephem-wheel-cache-t_hqlk2_/wheels/8c/e1/21/cbf69ad29fcb0b6cdd027f5ecac89616cafb959e36f5991c4c
Successf

In [ ]:
!pip install stable_baselines3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import rsoccer_gym
import gymnasium as gym

import stable_baselines3
from stable_baselines3 import DDPG
from stable_baselines3.common.logger import configure
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.noise import OrnsteinUhlenbeckActionNoise

import numpy as np

In [ ]:
# Create the environment and initialize it with reset method
env = gym.make('VSS1vs1-v0')
env.reset()

# Set up logger to store episode mean reward value in a csv file
csv_path = "./sb3_log"
custom_logger = configure(
    folder=csv_path,
    format_strings=["stdout", "csv"]
)

# Create a callback to evaluate the agent every eval_freq steps
eval_env = gym.make('VSS1vs1-v0')
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path="./eval_logs",
    log_path="./eval_logs",
    n_eval_episodes=10,
    eval_freq=100_000,
    deterministic=True,
    render=False,
    verbose=1
)

# Create an action noise object and add it to the model
n_actions = env.action_space.shape[-1]
action_noise = OrnsteinUhlenbeckActionNoise(
    mean=np.zeros(n_actions),
    sigma=0.1 * np.ones(n_actions)
)

# Create the model, set the new logger and train
model = DDPG(
    "MlpPolicy",
    env,
    learning_rate=6.25e-4,
    action_noise=action_noise,
    verbose=1
)
model.set_logger(custom_logger)
model.learn(total_timesteps=1_000_000, callback=eval_callback)

Environment initialized
Logging to ./sb3_log


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Environment initialized
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.2e+03  |
|    ep_rew_mean     | -15.1    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 139      |
|    time_elapsed    | 34       |
|    total_timesteps | 4800     |
| train/             |          |
|    actor_loss      | -2.04    |
|    critic_loss     | 0.0029   |
|    learning_rate   | 0.000625 |
|    n_updates       | 4699     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.2e+03  |
|    ep_rew_mean     | -7.87    |
| time/              |          |
|    episodes        | 8        |
|    fps             | 141      |
|    time_elapsed    | 67       |
|    total_timesteps | 9600     |
| train/             |          |
|    actor_loss      | -6.6    

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=100000, episode_reward=7.82 +/- 32.83
Episode length: 1133.20 +/- 200.40
---------------------------------
| eval/              |          |
|    mean_ep_length  | 1.13e+03 |
|    mean_reward     | 7.82     |
| time/              |          |
|    total_timesteps | 100000   |
| train/             |          |
|    actor_loss      | -55.1    |
|    critic_loss     | 0.913    |
|    learning_rate   | 0.000625 |
|    n_updates       | 99899    |
---------------------------------
New best mean reward!
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.14e+03 |
|    ep_rew_mean     | -7.42    |
| time/              |          |
|    episodes        | 92       |
|    fps             | 138      |
|    time_elapsed    | 755      |
|    total_timesteps | 104693   |
| train/             |          |
|    actor_loss      | -53.4    |
|    critic_loss     | 0.755    |
|    learning_rate   | 0.000625 |
|    n_updates       | 104592   |
--